In [2]:
import pandas as pd
import os

# Function to get the latest CSV file in the directory
def get_latest_csv(directory):
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    latest_file = max(csv_files, key=lambda x: os.path.getmtime(os.path.join(directory, x)))
    return latest_file

# Directory containing the CSV files
directory_path = './'  # Adjust to your directory path

# Get the latest CSV file
latest_csv = get_latest_csv(directory_path)

# Load the latest CSV file
df = pd.read_csv(latest_csv, encoding='utf-8')

# Mapping keywords into parts
keywords = {
    'Building': [],
    'Phase': ['Phase', 'Site', '(Block'],
    'Tower/Block': ['Tower', 'Block', 'Path', 'Street', 'Browdway',
                    'Celestial Avenue', 'Carpark', 'Road', 'Lane',
                    'Drive', 'Vista Avenue', 'Mansion', 'House',
                    'Sky One'],
    'Floor': ['Floor', '/F', 'LG', 'B/M', 'U/L', 'UCP', 'P3', 'P2',
              'Podium', 'PODIUM', 'U/G', 'P7', 'P4', 'UG1'],
    'Flat': ['Flat', 'Apartment', 'Duplex']
}

floor_types = ['Upper Floor', 'Middle Floor', 'Lower Floor']

# Classify parts with structured backward filling correctly targeting the rightmost columns
def classify_parts(parts):
    if len(parts) == 5:
        return parts + [None]  # Adding None for Floor_Type
    else:
        structured = {key: None for key in ['Building', 'Phase', 'Tower/Block', 'Floor', 'Flat', 'Floor_Type']}
        structured['Building'] = parts[0]  # Assign the first part to Building
        
        used = [False] * len(parts)
        used[0] = True  # First part is already used for Building
        
        # Identify and assign parts by keywords and floor types
        for i, part in enumerate(parts):
            if i == 0:
                continue
            if any(ft in part for ft in floor_types):
                structured['Floor_Type'] = part
                used[i] = True
                
            for key, hints in keywords.items():
                if any(hint in part for hint in hints):
                    structured[key] = part
                    used[i] = True
                    break
                
        # Backward filling for unused parts
        unused_parts = [parts[i] for i in range(len(parts)) if not used[i]]
        fill_order = ['Flat', 'Floor', 'Tower/Block', 'Phase']
        
        for key in fill_order:
            if unused_parts and structured[key] is None:
                structured[key] = unused_parts.pop(0)
        
        return [structured[key] for key in ['Building', 'Phase', 'Tower/Block', 'Floor', 'Flat', 'Floor_Type']]
    
# Apply the function to Address column and expand to new structure
df[['Building', 'Phase', 'Tower/Block', 'Floor', 'Flat', 'Floor_Type']] = (
    df['Address'].apply(lambda x: classify_parts(x.split('・'))).apply(pd.Series)
)

# Generate output file name based on input file name
output_file_name = latest_csv.replace('.csv', '_lv_2.csv')

# Output and save the result
df.to_csv(output_file_name, index=False)
df.head()


,Date,Address,Price,PriceTag,Area,Ft_Price,Agency,Region,District,Subdistrict,Code,Area_URL,Building,Phase,Tower/Block,Floor,Flat,Floor_Type
0,2025-02-20,Shun Fai Building・Lower Floor・Flat A,$4.72M,S,323ft²,"@$14,613",Centaline,Hong Kong Island,All Hong Kong Island,Kennedy Town,HMA111,https://hk.centanet.com/findproperty/en/list/t...,Shun Fai Building,None,None,Lower Floor,Flat A,Lower Floor
1,2025-02-19,Smithfield Terrace・Block A・Lower Floor・Flat 4,$3.3M,S,265ft²,"@$12,453",Centaline,Hong Kong Island,All Hong Kong Island,Kennedy Town,HMA111,https://hk.centanet.com/findproperty/en/list/t...,Smithfield Terrace,None,Block A,Lower Floor,Flat 4,Lower Floor
2,2025-02-18,Harbour View Garden・Tower 3・Lower Floor・Flat A,"$19,000",L,405ft²,@$47,Centaline,Hong Kong Island,All Hong Kong Island,Kennedy Town,HMA111,https://hk.centanet.com/findproperty/en/list/t...,Harbour View Garden,None,Tower 3,Lower Floor,Flat A,Lower Floor
3,2025-02-17,Serene Court・21/F・Flat C,$10.8M,S,691ft²,"@$15,630",Land Registry,Hong Kong Island,All Hong Kong Island,Kennedy Town,HMA111,https://hk.centanet.com/findproperty/en/list/t...,Serene Court,None,None,21/F,Flat C,None
4,2025-02-15,Kennedy 38・Upper Floor・Flat L,"$23,000",L,302ft²,@$76,Centaline,Hong Kong Island,All Hong Kong Island,Kennedy Town,HMA111,https://hk.centanet.com/findproperty/en/list/t...,Kennedy 38,None,None,Upper Floor,Flat L,Upper Floor
